In [62]:
from mlrun.run import new_function
from mlrun import get_or_create_ctx
from mlrun import mount_v3io,NewTask

In [63]:
docker_registry = 'docker-registry.default-tenant.app.mdl1222.iguazio-cd2.com:80'

In [64]:
#build a new image with all the depending libraries
# One time or to refresh image
#from mlrun import builder
#builder.build_image('spark/mlrun',base_image='urihoenig/spark-app-25:b2782')

In [65]:
# initialize context
###context = get_or_create_ctx('train')

In [66]:
#fn = new_function(command='spark://marcelo/sparkk8s#local:///User/spark-jdbc.py')
pyscript = "/v3io/users/admin/mlrun_demos/spark/train.py"
pyscript = "/User/mlrun_demos/spark/train.py"

#pyscript = context.get_param('train_data',pyscript)

In [67]:
#fn = new_function(name='iris-trainer',command='spark://urihoenig/spark-app:2.4.4-2.9.0-0.0.2#local://'+ pyscript)
fn = new_function(kind='spark',name='iris-trainer',command='urihoenig/spark-app:2.4.4-2.9.0-0.0.1#local://'+ pyscript)
fn.with_limits(mem="512m")
fn.with_requests(cpu=1)

In [80]:
fn.interactive = True   # Watch job execution
#fn.with_requests(cpu=1)
fn.spec.rundb = 'http://mlrun-api:8080'
#fn.spec.rundb = '/User/mlrun-db'
fn.with_igz_spark('2.5_b21_20191209191923')
#sj.with_igz_spark()


In [81]:
fn.spec.deps = {'jars': ['/igz/java/libs/v3io-hcfs_2.11-2.5_b21_20191209191923.jar',
                     '/igz/java/libs/v3io-spark2-streaming_2.11-2.5_b21_20191209191923.jar',
                     '/igz/java/libs/v3io-spark2-object-dataframe_2.11-2.5_b21_20191209191923.jar',
                     '/igz/java/libs/scala-library-2.11.12.jar'],
            'files': ['/igz/java/libs/v3io-py-2.5_b21_20191209191923.zip']}

In [82]:
fn.apply(mount_v3io())

In [83]:
print(fn.to_yaml())


kind: spark
metadata:
  name: iris-trainer
  project: default
spec:
  command: local:///User/mlrun_demos/spark/train.py
  args: []
  image: urihoenig/spark-app:2.4.4-2.9.0-0.0.1
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: d3290ed7-c878-4eb5-a192-a35d53f5e878
        container: users
        subPath: /admin
    name: v3io
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: d3290ed7-c878-4eb5-a192-a35d53f5e878
        container: ''
        subPath: ''
    name: v3io-fuse
  - hostPath:
      path: /dev/shm/default-tenant
      type: ''
    name: shm
  - hostPath:
      path: /var/run/iguazio/dayman/default-tenant
      type: ''
    name: v3iod-comm
  - emptyDir: {}
    name: daemon-health
  - configMap:
      defaultMode: 420
      name: spark-operator-v3io-config
    name: v3io-config
  volume_mounts:
  - mountPath: /User
    name: v3io
  - mountPath: /v3io
    name: v3io-fuse
  - mountPath: /dev/shm
    name: shm
  - mountPa

In [84]:
task = NewTask(name='iris-trainer',out_path='/User/mlrun-data/data').with_params(train_data="/User/mlrun_demos/spark/data/iris.csv").with_hyper_params({"regularization_rate":[0.01,0.02,0.03],},'max.accuracy')

In [85]:
print(task.to_yaml())

spec:
  parameters:
    train_data: /User/mlrun_demos/spark/data/iris.csv
  hyperparams:
    regularization_rate:
    - 0.01
    - 0.02
    - 0.03
  selector: max.accuracy
  outputs: []
  output_path: /User/mlrun-data/data
  secret_sources: []
metadata:
  name: iris-trainer
  iteration: 0



In [86]:
fn.run(task)

[mlrun] 2019-12-30 11:31:26,211 starting run iris-trainer uid=29caf9eb18644db4a03d028a75044d91  -> http://mlrun-api:8080


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...044d91,0,Dec 30 11:31:26,error,iris-trainer,kind=sparkowner=admin,,train_data=/User/mlrun_demos/spark/data/iris.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 29caf9eb18644db4a03d028a75044d91  , !mlrun logs 29caf9eb18644db4a03d028a75044d91 
[mlrun] 2019-12-30 11:32:56,401 run executed, status=error
runtime error: POST http://mlrun-api:8080/api/submit, error: HTTPConnectionPool(host='mlrun-api', port=8080): Read timed out. (read timeout=90)


RunError: POST http://mlrun-api:8080/api/submit, error: HTTPConnectionPool(host='mlrun-api', port=8080): Read timed out. (read timeout=90)